In [ ]:
# загружаем данные
from os import path
import pandas as pd
import numpy as np

data_dir = "/home/ubuntu/data/lastfm/lastfm-dataset-360K"

col_types = {
    "user": str,
    "artist-mbid": str,
    "artist-name": str,
    "total-plays": str
}
data = pd.read_csv(
    path.join(data_dir, "usersha1-artmbid-artname-plays.tsv"),
    sep="\t",
    header=None,
    names=col_types.keys(),
    dtype=col_types
)
data.head()

In [3]:
data.describe()

,total-plays
count,1.753566e+07
mean,2.151932e+02
std,6.144815e+02
min,0.000000e+00
25%,3.500000e+01
50%,9.400000e+01
75%,2.240000e+02
max,4.191570e+05


In [4]:
# заполняем пустые значения строкой None
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,37426
1,1099,1,152041
2,897,1,112367
3,717,1,38435
4,706,1,117444


In [5]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(data.index.values, replace=False, size=int(data.index.max() * 0.2))
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [6]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 9
нет в тестовой выборке, но есть в обучающей: 368
всего пользователей: 358868


In [7]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
test_data.drop(test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)
train_data.drop(train_data[train_data["user_id"].isin(user_ids_to_exclude).values].index, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)

In [9]:
train_file_name = "train/lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [11]:
# создаём разреженную матрицу user*item
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

In [12]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender()
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 6.915686368942261 секунд


In [13]:
print("получаем рекомендации для всех пользователей")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 305.62073278427124 секнуд


In [14]:
# используем mrec в качестве метрики качества
!DATA_DIR=/home/ubuntu/data/lastfm/lastfm-dataset-360K mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs

[2017-11-11 14:05:58,258] INFO: processing /home/inpefess/Downloads/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.1328 +/- 0.0000
prec@5         0.0490 +/- 0.0000
prec@10        0.0396 +/- 0.0000
prec@15        0.0264 +/- 0.0000
prec@20        0.0198 +/- 0.0000
